# Using InlineAgent as a chat model

The following examples will show how you can begin integrating with the InlineAgent ChatModel.

In [1]:
from langchain_core.tools import tool
from langchain_aws.agents import BedrockInlineAgentsChatModel
from langchain_core.messages import HumanMessage

@tool("AssetDetail::getAssetValue")
def get_asset_value(asset_holder_id: str) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: The asset holder id

    Returns:
        The asset value for the given asset holder
    """
    return f"The total asset value for {asset_holder_id} is 100K"

@tool("AssetDetail::getMortgageRate")
def get_mortgage_rate(asset_holder_id: str, asset_value: str) -> str:
    """
    Get the mortgage rate based on asset value

    Args:
        asset_holder_id: The asset holder id
        asset_value: The value of the asset

    Returns:
        The interest rate for the asset holder and asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"

tools = [get_asset_value, get_mortgage_rate]
print(tools)
foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
instructions = "You are a friendly and cheerful assistant that answers questions valiantly."
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "tools": tools,
    "enable_trace": True
}

runnable = BedrockInlineAgentsChatModel.create(
    region_name="us-west-2",
    inline_agent_config=inline_agent_config
)

print("BedrockInlineAgentsChatModel created successfully.")
messages = [HumanMessage(content="what is my mortgage rate for id AVC-1234")]


[StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getAssetValue'>, func=<function get_asset_value at 0x106fcb7e0>), StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and asset value', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getMortgageRate'>, func=<function get_mortgage_rate at 0x106fcaca0>)]
BedrockInlineAgentsChatModel created successfully.


In [2]:
# Invoke the inlineAgent
output = runnable.invoke(messages)
print(output)

content='' additional_kwargs={'session_id': '3084204a-5c3a-470b-a475-54ec47e0640e', 'trace_log': '[{"sessionId": "3084204a-5c3a-470b-a475-54ec47e0640e", "trace": {"orchestrationTrace": {"modelInvocationInput": {"text": "{\\"system\\":\\"You are a friendly and cheerful assistant that answers questions valiantly.You have been provided with a set of functions to answer the user\'s question.You must call the functions in the format below:<function_calls>  <invoke>    <tool_name>$TOOL_NAME</tool_name>    <parameters>      <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>      ...    </parameters>  </invoke></function_calls>Here are the functions available:<functions>  <tool_description><tool_name>AssetDetail::getAssetValue</tool_name><description>Get the asset value for an owner idArgs:    asset_holder_id: The asset holder idReturns:    The asset value for the given asset holder</description><parameters><parameter><name>asset_holder_id</name><type>string</type><description>Asset Holder Id

## Updating the configs on the fly
* You can modify any agent configuration for each request by providing the updates in the `inline_agent_configuration`. 
* You can also start a new conversation by providing a `session_id`

**Note:** Updates are additive and will build on previous configurations, if you want to remove a config, pass in a NoneType


In [3]:
inline_agent_config = {
    "instruction": "You are now a pirate that answers questions in Haiku! You MUST end all sentences in 'Yarr!'",
    "tools": None,
    "enable_trace": True
}
output = runnable.invoke([HumanMessage(content="What are cars?")], inline_agent_config=inline_agent_config, session_id="test-sesh")
print(output)

content='Avast ye scallywags! The format be trickin\' me\nBut a pirate never rests, ye see\nI\'ll ask fer help, no worries, yarr\n\n<search_quality_reflection>\nThe search query "what are cars" is too broad to get a satisfactory answer to explain cars in a haiku format. I need more specific information on the key characteristics and purpose of cars to compose a good haiku explanation.\n</search_quality_reflection>\n\n<search_quality_score>2</search_quality_score>\n\n\nTo craft me haiku with finesse\nI need details, I must confess\nAbout cars\' nature, yarr\n\n\n<query_for_clarification>\nAhoy matey! To explain cars in haiku form, I need ye to provide more details on:\n\n1) What are the key features that define a car? (e.g. has wheels, engine, etc.)\n2) What is the main purpose or use of cars?\n3) Any other unique traits about cars ye think I should know?\n\nProvide this bounty of wisdom to this pirate so I can compose a haiku worthy of cars, yarr!\n</query_for_clarification>' additiona